In [2]:
import os
import shutil
import subprocess
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import tqdm

from scipy.stats import kurtosis
from scipy.spatial import ConvexHull
import math
import random
import statistics as stats
from itertools import chain
from itertools import islice 
import numpy as np

import pandas as pd
from shapely.geometry import MultiPoint
import geopandas as gpd
from shapely.geometry import Polygon
import laspy

import utils.GapFraction_utils
from utils.gtqc_descriptors import extract_geometric_features_in_parallel, process_tree_list
from utils.gtqc_utils import *

from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix as cm
from sklearn.preprocessing import MinMaxScaler

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
# The directory for the new GT QC boxes that you want to add for the model
gtqc_FinalDir = r"" # Dir == Directory

vaPrefixPattern = "" # The Prefix assigned to each GT Box
#lazType = ".las" # The LAZ/LAS type for the files that exist within the GT boxes
completedGT = False # True : The GT has been QC'ed and we are reviewing the model / False : The GT has not been QC'ed and we are running the model

# Only complete this section if running VA stems that are not in boxes
va_GT = True # False :  The data is all in GT boxes for review / True : The data is just in a single shapefile to download the LAZ and clip it.
va_merged_stems = r"" # VA merged stems shapefile path
productionPolys = r"" # Production polygons shapefile path
s3_lazDir = "" # Include the final '/' at the end of the S3 directory path


# The directory containing all training data
allStems_df_trainingDir = r"" # Dir == Directory

# Locate the LasToLocalCoords EXE file
LasToLocalCoords = r"" # Path to LasToLocalCoords.exe

# Step 1 : Preprocessing
## Access and prepare new GT stems

In [ ]:
if va_GT == False:
    gtqc_finishedStemCollect(gtqc_FinalDir, vaPrefixPattern, completedGT)
if va_GT == True:
    va_laz_to_local(va_merged_stems, productionPolys, s3_lazDir, gtqc_FinalDir)
LASandSHP_prep(gtqc_FinalDir,completedGT)
LasFromCoords(gtqc_FinalDir,completedGT,LasToLocalCoords)

--- Processing LasFromCoords ---


KeyboardInterrupt: 

## Step 2 : Prepare test model descriptors
### Produce voxel descriptors

In [2]:
# Make sure you add the labels CSV file in the LAS directory, labelling whether it's an approved stem or rejected
if completedGT == True:
    folderPaths = [os.path.join(gtqc_FinalDir,"voxel", 'rejected'),os.path.join(gtqc_FinalDir,"voxel", 'approved')]
if completedGT == False:
    folderPaths = [os.path.join(gtqc_FinalDir,"voxel")]

for folder in folderPaths: # Iterate through the folders
    # Get list of TXT files
    txtFiles = []
    for root,dirs,_ in os.walk(folder):
        for name in dirs:
            if 'voxel' in os.path.basename(name):
                continue
            txtFiles.append(os.path.join(root,name,".txt"))

    # Process all trees voxel fractions in parallel
    flattened_df = process_tree_list(txtFiles, run_in_parallel=False)
    print("Completed processing all trees")
    print(flattened_df.shape)

    # Write all dictionaries to a TXT file
    os.makedirs(os.path.join(folder,"voxel"),exist_ok=True)

    if completedGT == True:
        # Get all labels data
        labelFiles = [os.path.join(folder,file) for file in os.listdir(os.path.join(folder)) if file.endswith(".csv")]
        label_list = [pd.read_csv(file) for file in labelFiles]
        combined_labels = pd.concat(label_list, ignore_index=True)  # Combine into a single DataFrame
        allStems_df = pd.merge(flattened_df,combined_labels,on="BOX",how='inner') # Merge labels with the flattened dataframe
    else:
        allStems_df = flattened_df
    allStems_df.to_csv(os.path.join(folder,"voxel","stems_NoLAZextracts.csv"),index=False) # Export the first output to CSV


NameError: name 'completedGT' is not defined

### LAZ Extract Descriptors

In [ ]:
# List of LAZ Extract columns
laz_extract_columnList = ["BBox_X","BBox_Y","BBox_Z","Var_X","Var_Y","Var_Z","pDensity","Centroid_X","Centroid_Y","Centroid_Z","Eig_X","Eig_Y","Eig_Z","Int_StD","Int_Avg","Int_Skew","Int_Kurt","CHull_1","CHull_2","CHull_3","CHull_4","CHull_5","CHull_6","CHull_7","CHull_8","CHull_9","CHull_10","NN_1","NN_2","NN_3","NN_4","NN_5","NN_6","NN_7","NN_8","NN_9","NN_10"]

# Make sure you add the labels CSV file in the LAS directory, labelling whether it's an approved stem or rejected
if completedGT == True:
    folderPaths = [os.path.join(gtqc_FinalDir,"voxel", 'rejected'),os.path.join(gtqc_FinalDir,"voxel", 'approved')]
if completedGT == False:
    folderPaths = [os.path.join(gtqc_FinalDir,"voxel")]

for folder in folderPaths:
    # Run this cell if you didn't get lasFiles earlier
    lasFiles = [os.path.join(folder,file) for file in os.listdir(folder) if file.endswith(".las")]

    # Extract features from training data
    train_features = []
    train_features = extract_geometric_features_in_parallel(lasFiles)

    # Add data to the full data frame
    laz_extract = pd.DataFrame(train_features,columns=laz_extract_columnList)
    lasBox = []
    for file in lasFiles: # Add box names column
        lasBox.append(file.split("\\")[-1].replace(".las",""))
    #print(lasBox)
    laz_extract["BOX"] = lasBox # Add box names column in order of lasFiles
    allStems_df = pd.concat([pd.read_csv(os.path.join(folder, "voxel",file)) for file in os.listdir(os.path.join(folder,"voxel")) if file.startswith("stems")]) # Access all CSVs in directory

    allStems_df = pd.merge(allStems_df,laz_extract,on="BOX",how='inner') # Merge with big data frame


    allStems_df.to_csv(os.path.join(folder,"voxel","all_stems.csv"),index=False) # Final output
    # Output to voxel/voxel folder or voxel/approved/voxel folder

## Step 3 : Collect Training Data

In [ ]:
allStems_df_training = pd.concat([pd.read_csv(os.path.join(allStems_df_trainingDir,file)) for file in os.listdir(allStems_df_trainingDir) if file.endswith(".csv")], ignore_index=True)#allStems_df#[allStems_df["BOX"].str[-3] == "d"]
# Rename the column 'Labels' to 'Label'
#allStems_df_training.rename(columns={'Labels': 'Label'}, inplace=True)

print(f"{allStems_df_training.shape}")

# Drop columns with NaN values and fill in those values with 0s - There shouldn't really be much of this anymore, except for the convex hull and nearest neighbour columns when there aren't enough points
# Fill NaN
allStems_df_training.fillna(0,inplace=True)
a = list(allStems_df_training.columns[allStems_df_training.isna().any()])
allStems_df_training = allStems_df_training.drop(columns=a)
allStems_df_training.fillna(0,inplace=True)

cols_to_drop = [a for a in list(allStems_df_training.columns) if '(' in a]
allStems_df_training = allStems_df_training.drop(cols_to_drop,axis=1)


# Convert all cols to numeric
# Iterate through each column and convert to numeric (except "BOX")
for col in allStems_df_training.columns:
    if col != "BOX":  # Skip the "BOX" column
        # Convert the column to numeric, coercing errors to NaN (if any)
        allStems_df_training[col] = pd.to_numeric(allStems_df_training[col], errors='coerce')

allStems_df_training = allStems_df_training.drop(["BBox_X","BBox_Y","BBox_Z","Centroid_X","Centroid_Y","Centroid_Z"],axis=1)#,'Unnamed: 0'],axis=1)

(27883, 1068)


## Step 4 : Modelling

In [7]:
# Drop unnecessary columns
nunique = allStems_df_training.nunique()
cols_to_drop = nunique[nunique == 1].index
allStems_df_training = allStems_df_training.drop(cols_to_drop, axis=1)

# Set up for modelling
allStems_df_Descriptors = allStems_df_training.drop(columns=["BOX","Label"])
allStems_df_Labels = allStems_df_training['Label']
allStems_df_BOX= allStems_df_training['BOX']

# Scaling the data based on minMax values 
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(allStems_df_Descriptors)
allStems_df_Descriptors_scaled = min_max_scaler.transform(allStems_df_Descriptors)

# Transform to arrays
#train_descriptors = np.array(allStems_df_Descriptors)
train_descriptors = np.array(allStems_df_Descriptors_scaled)
train_labels = np.array(allStems_df_Labels)

# Model
testSize = 0.10
#svm_classifier = SVC(kernel="poly",probability=True,gamma=0.0001,C=100)

classifiers = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=0),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
    "HistGradientBoosting": HistGradientBoostingClassifier(max_iter=1000),
    "XGBoost": XGBClassifier(objective="binary:logistic", random_state=42)
}

## UNCOMMENT the following line to copy over the best classifier from the last run
classifiers = {"HistGradientBoosting": HistGradientBoostingClassifier(max_iter=1000)} # copy over the winner classifier here (do nothing if the last one is the best)


for name, clf in classifiers.items():
    print(f"Training {name} classifier...")
    X_train, X_test, y_train, y_test = train_test_split(train_descriptors, train_labels, test_size=testSize, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"Results for {name} classifier:")
    evaluate_model(y_test, y_pred)
    print("\n")

Training HistGradientBoosting classifier...
Results for HistGradientBoosting classifier:
The model's accuracy is 77.38%
-------------------
--------  Rej. App.  Precision
Rejected [717 372]    0.66
Approved [ 259 1441]    0.85
Recall   0.73  0.79
-------------------
Rejected F1 Score: 0.694
Approved F1 Score: 0.820




In [9]:
# Grab the testing data
if completedGT == True:
    allStemsFiles = [pd.read_csv(os.path.join(gtqc_FinalDir,"voxel",ar,"voxel",file)) for ar in ["approved","rejected"] for file in os.listdir(os.path.join(gtqc_FinalDir,"voxel",ar,"voxel")) if file.endswith("stems.csv")]
    allStems_df = pd.concat(allStemsFiles)
    allStems_testing_Labels = allStems_df['Label']
    #testing_labels = np.array(allStems_testing_Labels)
else:
    allStemsFiles = [pd.read_csv(os.path.join(gtqc_FinalDir,"voxel","voxel",file)) for file in os.listdir(os.path.join(gtqc_FinalDir,"voxel","voxel")) if file.endswith("stems.csv")]
    allStems_df = pd.concat(allStemsFiles)

allStems_df = pd.concat(allStemsFiles)
print(allStems_df.shape)
allStems_testing_BOX= allStems_df['BOX']

# Drop unnecessary columns
nunique = allStems_df.nunique()
cols_to_drop = nunique[nunique == 1].index
allStems_df_subset = allStems_df.drop(cols_to_drop, axis=1)

# Get the intersection of column names from both DataFrames
common_columns = allStems_df_subset.columns.intersection(allStems_df_Descriptors.columns)
# Keep only the columns in df1 that are also in df2
allStems_df_subset = allStems_df_subset[common_columns]

# Set up for modelling
if list(allStems_df_subset.columns) == list(allStems_df_Descriptors.columns):
    print("The columns are identical between datasets")
else:
    print(allStems_df_subset.columns)
    print(allStems_df_Descriptors.columns)
allStems_testing_Descriptors = allStems_df_subset#.drop(columns=["BOX"])#,"Labels"])

allStems_testing_Descriptors_scaled = min_max_scaler.transform(allStems_testing_Descriptors) # Use training set minMax values to scale the testing data

# Transform to arrays
#testing_descriptors = np.array(allStems_testing_Descriptors)
testing_descriptors = np.array(allStems_testing_Descriptors_scaled)
print(testing_descriptors)


(25251, 1067)
The columns are identical between datasets
[[0.         0.         0.         ... 0.26183769 0.22297313 0.        ]
 [0.         0.         0.         ... 0.32218884 0.25245244 0.12444728]
 [0.         0.         0.         ... 0.2089347  0.30611223 0.22815684]
 ...
 [0.         0.         0.         ... 0.15466364 0.14103516 0.10066444]
 [0.01162791 0.         0.         ... 0.09527896 0.19153069 0.13460381]
 [0.         0.         0.         ... 0.12739115 0.16027083 0.08213523]]


# Step 5 : Implementing the model
#### 5a: Test the model results on either the GT Boxes or the testing set put aside from the training set

In [6]:
threshold = 0.68

if completedGT == True:
    print("--- Testing the model on completed GT boxes ---")
    print("")
    test_proba = clf.predict_proba(testing_descriptors)

    within_thresh_dict = dict()
    correct_stems = []
    incorrect_stems = []
    for i, stem_prob in enumerate(test_proba):
        if stem_prob[1] > threshold:
            within_thresh_dict[f"STEM{i}"] = [list(allStems_testing_BOX)[i],list(allStems_testing_Labels)[i]]
            if list(allStems_testing_Labels)[i] == 0:
                incorrect_stems.append(list(allStems_testing_BOX)[i])
            else:
                correct_stems.append(list(allStems_testing_BOX)[i])
            print(f"Stem {list(allStems_testing_BOX)[i]} (label: {list(allStems_testing_Labels)[i]}) probabilities: {stem_prob[1]}")

    # Calculate the proportion within a threshold
    within_thresh_labels = []
    for _,val in within_thresh_dict.items():
        within_thresh_labels.append(val[1])

    print(f"Incorrect stems: {incorrect_stems}")
    print("")
    print(f"{sum(within_thresh_labels)/len(within_thresh_labels)*100:.2f}% are correctly called above a {threshold*100:.2f}% threshold. Total stems = {len(within_thresh_labels)} out of {len(allStems_df_subset)}")
else:
    print(f"--- Testing the model on {testSize:.2f}% set aside from training set ---")
    print("")
    testing_proba = clf.predict_proba(X_test)

    within_thresh_dict = dict()
    correct_stems = []
    incorrect_stems = []
    for i, stem_prob in enumerate(testing_proba):
        if stem_prob[1] > threshold:
            within_thresh_dict[f"STEM{i}"] = [i,list(y_test)[i]]
            if list(y_test)[i] == 0:
                incorrect_stems.append(i)
            else:
                correct_stems.append(i)
            print(f"Stem {i} (label: {list(y_test)[i]}) probabilities: {stem_prob[1]}")

    # Calculate the proportion within a threshold
    within_thresh_labels = []
    for _,val in within_thresh_dict.items():
        within_thresh_labels.append(val[1])

    print(f"Incorrect stems: {incorrect_stems}")
    print("")
    print(f"{sum(within_thresh_labels)/len(within_thresh_labels)*100:.2f}% are correctly called above a {threshold*100:.2f}% threshold. Total stems = {len(within_thresh_labels)} out of {len(X_test)}")

NameError: name 'testSize' is not defined

#### 5b: Implement the model on new data
##### First test it on a small subset of trees

In [3]:
# Use this cell to test the model before implementing it
max_num = 100 # The number of stems to select for reviewing


# Read the full shapefile and extract only UNIQUE_ID and STEREO_SPP columns
stems_with_uniqueIDandSpecies = gpd.read_file(va_merged_stems)
stems_with_uniqueIDandSpecies.columns = stems_with_uniqueIDandSpecies.columns.str.upper()
stems_with_uniqueIDandSpecies = stems_with_uniqueIDandSpecies[["UNIQUE_ID","STEREO_SPP"]]

if completedGT == False:
    test_proba = clf.predict_proba(testing_descriptors)
    os.makedirs(os.path.join(gtqc_FinalDir,"model_testing"),exist_ok=True)

    within_thresh_dict = dict()
    selected_stems = []
    for i, stem_prob in enumerate(test_proba):
        if stem_prob[1] > threshold:
            within_thresh_dict[f"STEM{i}"] = [list(allStems_testing_BOX)[i]]
            selected_stems.append({list(allStems_testing_BOX)[i]: stem_prob[1]})
            print(f"Stem {list(allStems_testing_BOX)[i]} (probabilities: {stem_prob[1]})")
    print(f"Total number of stems selected: {len(selected_stems)} out of {len(allStems_df_subset)} | {len(selected_stems)/len(allStems_df_subset)*100:.2f}%")

    # Create a lookup dictionary from the DataFrame
    id_to_species = dict(zip(stems_with_uniqueIDandSpecies['UNIQUE_ID'], stems_with_uniqueIDandSpecies['STEREO_SPP']))
    # Build new dictionary using the selected_trees keys
    species_dict = {list(stem.keys())[0]: id_to_species.get(list(stem.keys())[0], "UNKNOWN") for stem in selected_stems}



    # Group stems by species
    species_groups = {}
    for stem_id, species in species_dict.items():
        if species not in species_groups:
            species_groups[species] = []
        species_groups[species].append(stem_id)

    # Randomly sample stems from each species group
    sampled_stems = []
    for species, stems in species_groups.items():
        num_to_sample = min(max_num // len(species_groups), len(stems))
        sampled_stems.extend(random.sample(stems, num_to_sample))
    sampled_stems_dict = {stem: next((s[stem] for s in selected_stems if stem in s), None) for stem in sampled_stems}
    print(sampled_stems_dict)
    
    #print(f"Randomly sampled stems across species: {sampled_stems[:20]}")

    with open(os.path.join(gtqc_FinalDir, "model_testing", "_selected_stems.txt"), "w") as f:
        for stem, prob in sampled_stems_dict.items():
            shutil.copy(
                os.path.join(gtqc_FinalDir, "voxel", stem + ".las"),
                os.path.join(gtqc_FinalDir, "model_testing", stem + ".las"),
            )
            f.write(f"{stem}: {prob}\n") 

print("-----------------------------------------------------------------------------------------------------------------------------")
# Determine the species distribution of the selected stems


#unique_sp = stems_with_uniqueIDandSpecies["STEREO_SPP"].unique()

total_counts = stems_with_uniqueIDandSpecies["STEREO_SPP"].value_counts()
selected_stems_fromFullSet = stems_with_uniqueIDandSpecies[stems_with_uniqueIDandSpecies["UNIQUE_ID"].isin([list(stem.keys())[0] for stem in selected_stems])]
selected_counts = selected_stems_fromFullSet["STEREO_SPP"].value_counts()

# Calculate proportions
proportions = {}
for species in total_counts.index:
    total = total_counts[species]
    selected = selected_counts.get(species, 0)
    proportions[species] = selected / total

# Print results
print(f"Proportion of each species included in the selected trees above the {threshold*100}% probability threshold:")
for species, proportion in proportions.items():
    print(f"{species}: {proportion:.2%}")

NameError: name 'gpd' is not defined

##### Then, review those point clouds externally (or by using these snapshots)

In [1]:
x_dim = 30

# Directory containing the LAS files
las_dir = os.path.join(gtqc_FinalDir,"model_testing")

# List all LAS files in the directory
las_files = [f for f in os.listdir(las_dir) if f.endswith('.las')]

# Plot each LAS file
for las_file in las_files:
    # Read the LAS file
    las = laspy.read(os.path.join(las_dir, las_file))
    
    # Extract the x, y, z coordinates
    x = las.x
    y = las.y
    z = las.z
    
    # Create a 3D plot with 4 subplots
    fig = plt.figure(figsize=(20, 5))
    
    # Different angles for the subplots
    angles = [(x_dim, 0), (x_dim, 90), (x_dim, 180), (x_dim, 270)]
    
    for i, angle in enumerate(angles):
        ax = fig.add_subplot(1, 4, i+1, projection='3d')
        ax.scatter(x, y, z, c=z, cmap='viridis', marker='.', s=4)
        
        # Set plot title and labels
        ax.set_title(f'{las_file.replace(".las","")} (Angle: {angle})')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        
        # Set the viewing angle
        ax.view_init(elev=angle[0], azim=angle[1])
    
    plt.show()

NameError: name 'os' is not defined

##### And finally, implement the model

In [4]:
if completedGT == False:
    # Use this cell to implement the model
    test_proba = clf.predict_proba(testing_descriptors)

    within_thresh_dict = dict()
    selected_stems = []
    for i, stem_prob in enumerate(test_proba):
        if stem_prob[1] > threshold:
            within_thresh_dict[f"STEM{i}"] = [list(allStems_testing_BOX)[i]]
            selected_stems.append(list(allStems_testing_BOX)[i])
            print(f"Stem {list(allStems_testing_BOX)[i]} (probabilities: {stem_prob[1]})")

    for stem in selected_stems:
        if va_GT == False:
            temp_dir = os.path.join(gtqc_FinalDir,stem[0:9],"trees")
            os.makedirs(os.path.join(temp_dir,"approved"),exist_ok=True)
            if os.path.isfile(os.path.join(temp_dir,stem+".las")):
                shutil.move(os.path.join(temp_dir,stem+".las"),os.path.join(temp_dir,"approved",stem+".las"))
            print(stem[0:9])

            # # Save the list of selected stems into voxel folder
            # selected_stems_df = pd.DataFrame(selected_stems,columns=["StemID"])
            # selected_stems_df.to_csv(os.path.join(gtqc_FinalDir,"Selected_Stems.csv"),index=False)
        else:
            os.makedirs(os.path.join(gtqc_FinalDir,"approved"),exist_ok=True)
            shutil.move(os.path.join(gtqc_FinalDir,"voxel",stem+".las"),os.path.join(gtqc_FinalDir,"approved",stem+".las"))

    # Save the list of selected stems into approved folder
    selected_stems_df = pd.DataFrame(selected_stems,columns=["StemID"])
    selected_stems_df.to_csv(os.path.join(gtqc_FinalDir,"Selected_Stems.csv"),index=False)


    print(f"{len(selected_stems)} were moved above a threshold of {threshold*100:.2f}%. Of a total of {len(allStems_df_subset)}, {len(selected_stems)/len(allStems_df_subset)*100:.2f}% were moved.")

NameError: name 'completedGT' is not defined